In [1]:
from scipy.spatial import distance as dist
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import imutils
import dlib
import cv2

In [2]:
def sound_alarm():
    #play an alarm sound
    playsound.playsound("analog-watch-alarm.wav")

In [3]:
#threshold euclidian distance between the farthest point of jaw and the mouth of either side of face
MOUTH_JAW_THRESH = 45.0
#number of consecutive frames
MOUTH_JAW_CONSEC_FRAMES = 10
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
COUNTER1 = 0
ALARM_ON = False

In [4]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/SAHIL/shape_predictor_68_face_landmarks.dat")

In [5]:
# grab the indexes of the facial landmarks for the mouth and jaw respectively
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]
(jStart, jEnd) = face_utils.FACIAL_LANDMARKS_IDXS["jaw"]

In [6]:
# start the video stream thread
cap = cv2.VideoCapture('1234.mp4')
    
if (cap.isOpened()== False):  
    print("Error opening video  file")    

# Read until video is completed 
while(cap.isOpened()): 
      
    #Capture frame-by-frame 
    ret, frame = cap.read() 
    
    if frame is None:
        break
     
    if ret == True:
        
        frame = imutils.resize(frame, width=450)
        gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        rects = detector(gray,0)

            # loop over the face detections
        for rect in rects:
            
                # determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array

                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                
                #extract coordinates of mouth and jaw
                
                mouth = shape[mStart:mEnd]
                jaw = shape[jStart:jEnd]
                
                #Left side and right side distance from eachother
                
                ldist = dist.euclidean(mouth[0], jaw[3])
                rdist = dist.euclidean(mouth[6], jaw[13])
            
                if (ldist < MOUTH_JAW_THRESH) or (rdist < MOUTH_JAW_THRESH) :
                    COUNTER1 += 1
                    if COUNTER1 >= MOUTH_JAW_CONSEC_FRAMES:

                        if not ALARM_ON:
                            ALARM_ON = True
                            t1 = Thread(target = sound_alarm())
                            t1.deamon = True
                            t1.start()
                        # draw an alarm on the frame
                        cv2.putText(frame, "DISTRACTION ALERT!", (100, 300),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                else:
                    COUNTER1 = 0
                    ALARM_ON = False
                #show mouth jaw distance on the frame run-time 
                cv2.putText(frame, "LMJD: {:.2f}".format(ldist), (0,30),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2 )
                cv2.putText(frame, "RMJD: {:.2f}".format(rdist), (0,60),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2 )
            # show the frame
        cv2.imshow("frame", frame)
        
        
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
            
    else:
        break
            
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()   